# Исследование данных о российском кинопрокате



Заказчик этого исследования — Министерство культуры Российской Федерации.
Данные взяты  из портала открытых данных Министерства культуры. Набор данных содержит информацию о прокатных удостоверениях, сборах и государственной поддержке фильмов, а также информацию с сайта КиноПоиск.

        
  > # Цель исследования
  Необходимо изучить рынок российского кинопроката и выявить текущие тренды. Уделить внимание фильмам, которые получили           государственную поддержку. Узнать насколько такие фильмы интересны зрителю.
  >> # Ход исследования
      - Обзор данных
      - Предобработка данных
      - Исследовательский анализ данных
      - Анализ фильмов с господдержкой
      - Общий вывод

### Шаг 1. Откройте файлы с данными и объедините их в один датафрейм. 

Объедините данные таким образом, чтобы все объекты из датасета `mkrf_movies` обязательно вошли в получившийся датафрейм. 

<div id="accordion">
    <div class="card">
        <div class="card-header" id="headingThree">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseHint_0" aria-expanded="false" aria-controls="collapseHint_0">Подсказка</button>
        </div>
        <div id="collapseHint_0" class="collapse" aria-labelledby="headingThree" data-parent="#accordion">
            <div class="card-body">
Обратите внимание на тип данных столбца, по которому будете соединять датафреймы. Тип данных этих столбцов должен быть одинаковым в обоих датафреймах.
            </div>
        </div>
    </div>
</div>

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
data_movies = pd.read_csv('/datasets/mkrf_movies.csv')
data_shows = pd.read_csv('/datasets/mkrf_shows.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/datasets/mkrf_movies.csv'

# Описание данных

 -   title — название фильма;
 -   puNumber — номер прокатного удостоверения;
 -   show_start_date — дата премьеры фильма;
 -   type — тип фильма;
 -   film_studio — студия-производитель;
 -   production_country — страна-производитель;
 -   director — режиссёр;
 -   producer — продюсер;
 -   age_restriction — возрастная категория;
 -   refundable_support — объём возвратных средств государственной поддержки;
 -   nonrefundable_support — объём невозвратных средств государственной поддержки;
 -    financing_source — источник государственного финансирования;
 -    budget — общий бюджет фильма;
 -    ratings — рейтинг фильма на КиноПоиске;
 -    genres — жанр фильма.
 -    puNumber — номер прокатного удостоверения;
 -    box_office — сборы в рублях.

In [ ]:
data_movies.head() #ознакомимся с первым файлом

In [ ]:
data_movies.info()

In [ ]:
data_movies['puNumber'].isna().sum() # при попытке слияния данных обнаружилось, что в столбце puNumber файла mkrf_movies.csv есть тип str, тогда как в другом файле тип даных столбца с аналогичным названием int

In [ ]:
data_movies = data_movies.query('puNumber != "нет"').reset_index(drop=True) # при попытке изменить тип на int появилась ошибка, из которой следовало что одна из ячеек содержет слово "нет".
# удаляем и сбрасываем индексы

In [ ]:
data_movies['puNumber'] = data_movies['puNumber'].astype('int') # меням тип данных


In [ ]:

data_shows #ознакомимся с вторым файлом


In [ ]:
df = data_movies.merge(data_shows, on = 'puNumber', how = 'left') # обьединяем два датасета в один датафрейм

In [ ]:
df

In [ ]:
df.info()

#### Результаты знакомства с данными
В результате знакомства с данными были выявлены ***пропуски*** в следйющих столбцах: 

film_studio 

production_country

director

producer
  
refundable_support 

nonrefundable_support 

budget 

financing_source 

ratings 

genres 

box_office

Также предварительно были выявлены ***несоответсвия типов данных*** в следйющих столбцах:

show_start_date

ratings

### Шаг 2. Предобработка данных

#### Шаг 2.1. Проверьте типы данных

- Проверьте типы данных в датафрейме и преобразуйте их там, где это необходимо.

In [ ]:
# более подробно рассмотрим типы данных
display(df.tail(10))
df.info()

In [ ]:

# следует заменить, как говорилось в обзоре данных, show_start_date на datetime, ratings на float  

df['show_start_date'] = pd.to_datetime(df['show_start_date'], format ='%Y-%m-%dT%H:%M:%S')



In [ ]:
df['ratings'].unique()

In [ ]:
# столбец ratings должен иметь числовой формат
# в этом столбце имеются строки, поэтому создадим функцию которая превратит рейтинг из процентов в десятичную оценку

def rating_convertor(i):

    if isinstance(i, str) and '%' in i:
         return float(i[:-1])/100
    return float(i)


In [ ]:
df['ratings'] = df['ratings'].apply(rating_convertor)

In [ ]:
df['ratings'].unique() 

##### Результаты проверки типов данных
В результате проверки типов данных были внесены измениния в столбцы: show_start_date на datetime, ratings на float.



#### Шаг 2.2. Изучите пропуски в датафрейме

- Объясните, почему заполнили пропуски определённым образом или почему не стали этого делать.

In [ ]:
# посмотрим какую долю в процентах занимают пропуски в каждом столбце
pd.options.display.float_format = '{:,.1f}'.format # отобразим резудьтат не в научном формате
pd.DataFrame(round(df.isna().mean()*100,)).style.background_gradient('coolwarm')

In [ ]:
# сразу видно что в столбце producer всего 8% пропусков, для экономии времени удалим эти строки.
df = df.dropna(subset = ['producer']).reset_index(drop=True)
pd.DataFrame(round(df.isna().mean()*100,)).style.background_gradient('coolwarm')

In [ ]:
df.query('ratings.isna() == True').head(10)
# оставим пропуски, так как заполнить их каким то явным занчением невозможно 

fundable_support, nonrefundable_support, budget, financing_source исмеют наибольший однаковый процент пропусков, однако известно что ти столбцы связаны с гос поддержкой и пропуски означают только то, что фильмы не поддердивались мин культуры и фондом кино. Их стоит оставить без измнения

Также пропуски в жанрах и рейтинге оставим без изменений


In [ ]:
df['year'] = pd.DatetimeIndex(df['show_start_date']).year

In [ ]:
df.query('box_office.isna()').pivot_table(index = 'year', values = 'type', aggfunc = 'count').plot(kind='bar', title = 'количество пропусков по годам')# пропуски по годам

##### Результат обработки пропусков

Были удалены пропуски из столбца producer (8% пропусков)

В остальных столбцах удаление пропусков или их замена невозможны

#### Шаг 2.3. Изучите дубликаты в датафрейме
- Проверьте, есть ли в данных дубликаты. Опишите причины, которые могли повлиять на появление дублей.

In [ ]:
for name,values in df.iteritems():
    if name in ['title','genres','financing_source','producer','director','production_country','film_studio','type']:
        df[name] = df[name].str.lower()
 
        

#Поиск явных дупликатов 
#Приведем строки к нижнему регистру

df['title'] = df['title'].str.lower()
df['genres'] = df['genres'].str.lower().str.strip()
df['financing_source'] = df['financing_source'].str.lower().str.strip()
df['producer'] = df['producer'].str.lower().str.strip()
df['director'] = df['director'].str.lower().str.strip()
df['production_country'] = df['production_country'].str.lower().str.strip()
df['film_studio'] = df['film_studio'].str.lower().str.strip()
df['type'] = df['type'].str.lower().str.strip()
df.duplicated().sum()

In [ ]:
df = df.replace('ё','е')

In [ ]:
df.duplicated().sum()

явных дупликатов не обнаружено


In [ ]:
df['type'].unique()

In [ ]:

df['film_studio'].unique()

На поиск неявных дубликатов в этом списке уйдет слишком много времени, поэтому оставим этот столбец без изменения. Стоит обратить внимание инженеров по сбору данных на представления информации в этом столбце.

In [ ]:
df['production_country']

In [ ]:
df['production_country'].unique()

In [ ]:
df['production_country'].nunique()

In [ ]:
# создадим функцию для того, чтобы заменить "-" на " "

def change(i):
    if isinstance(i, str) and "-" in i:
        return i.replace('-', ' ')
    return i    

In [ ]:
df['production_country'] = df['production_country'].apply(change)


In [ ]:
df['production_country'].unique()

In [ ]:
df['production_country'].nunique()

In [ ]:
df['director'].nunique()

In [ ]:
df['director']

In [ ]:
def dir_name(i):
    if isinstance(i, str) and "." in i[1] :
        return i[2:]
    return i    

In [ ]:
df['director'] = df['director'].apply(dir_name)

In [ ]:
df['producer'].str.split(', ').str[1]

In [ ]:
df['director'].nunique()

Нашли 77 дубликатов в этом столбце

In [ ]:
df['title'].nunique()

In [ ]:
df['title'].value_counts().head(20)

In [ ]:
# избавимся от спецсимволов в конце названий

def title(i):
    if "/" in i[-1] or "." in i[-1] or "_" in i[-1]  :
        return i.strip()[:-1]
    return i

In [ ]:
# избавимся от поясняющих записей типа " .../по роману..."
def title_find(i):
    if "/" in i:      
        index = i.find('/')
        return i[:index]  
                         
    return i

In [ ]:
df['title'] = df['title'].apply(title)

In [ ]:
df['title'].value_counts().head(20)

In [ ]:
df['title'] = df['title'].apply(title_find)

In [ ]:
df['title'].value_counts().head(20)

In [ ]:
df['title'].nunique()

In [ ]:
df.query('title == "день святого валентина"')

In [ ]:
df.duplicated().sum()

Дубликаты в столбце title нельзя удалять, т к у них разные прокатные удостоверения

In [ ]:
df

##### Результаты поиска дубликатов

Явных дубликатов обнаружено не было

При поиске неявных дубликатов было обнаружено множество опечаток

#### Шаг 2.4. Изучите категориальные значения

- Посмотрите, какая общая проблема встречается почти во всех категориальных столбцах;
- Исправьте проблемные значения в поле `type`.

<div id="accordion">
    <div class="card">
        <div class="card-header" id="headingThree">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseHint_1" aria-expanded="false" aria-controls="collapseHint_1">Подсказка</button>
        </div>
        <div id="collapseHint_1" class="collapse" aria-labelledby="headingThree" data-parent="#accordion">
            <div class="card-body">
В поле <code>type</code> есть несколько значений, у которых появился пробел в начале строки. Самый простой способ их «починить» -- использовать метод <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.str.strip.html#pandas.Series.str.strip">.str.strip</a>. Этот метод удаляет все пробелы, которые встречаются в начале и в конце строки. Применяют его таким образом:<br>
<code>df['type'].str.strip()</code>
            </div>
        </div>
    </div>
</div>

##### Результаты изучения категориальных значений
В категориальных переменных встречаются значения с пробелами в начале или конце строки, мы удалили их ранее

#### Шаг 2.5. Проверьте количественные значения

- Проверьте, обнаружились ли в таких столбцах подозрительные данные. Как с такими данными лучше поступить?

<div id="accordion">
    <div class="card">
        <div class="card-header" id="headingThree">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseHint_budget" aria-expanded="false" aria-controls="collapseHint_budget">Подсказка</button>
        </div>
        <div id="collapseHint_budget" class="collapse" aria-labelledby="headingThree" data-parent="#accordion">
            <div class="card-body">
Обратите внимание на описание столбца <code>budget</code>. Как этот столбец соотносится с двумя другими: <code>refundable_support</code> и <code>nonrefundable_support</code>?
            </div>
        </div>
    </div>
</div>

In [ ]:
# Подозрительные значения budget = 0, хотя господдержка включена в это значение и refundable_support и nonrefundable_support не равны 0
df.query('budget < refundable_support + nonrefundable_support ')

In [ ]:
df.query('box_office < 10000 ')

В данном столбце есть подозрительно маленькие значения, возможно здесь есть ошибка в разрядности или переводе в рубли. Такие подозрения из за наличия в этом списке именитых фильмов, кассовые сборы которых никак не могут быть маленькими.

Пока оставим эти значения, так как неизвестно из за чего они появились и пригодятся ли в дальнейшем

In [ ]:
df

##### Результаты изучения количественых значений
В данном столбце есть подозрительно маленькие значения, возможно здесь есть ошибка в разрядности или переводе в рубли. Такие подозрения из за наличия в этом списке именитых фильмов, кассовые сборы которых никак не могут быть маленькими.

Пока оставим эти значения, так как неизвестно из за чего они появились и пригодятся ли в дальнейшем

#### Шаг 2.6. Добавьте новые столбцы





- Создайте столбец с информацией о годе проката. Выделите год из даты премьеры фильма.

Ранее уже добавили этот столбец

- Создайте два столбца: с именем и фамилией главного режиссёра и основным жанром фильма. В столбцы войдут первые значения из списка режиссёров и жанров соответственно.

<div id="accordion">
    <div class="card">
        <div class="card-header" id="headingThree">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseHint_2" aria-expanded="false" aria-controls="collapseHint_2">Подсказка</button>
        </div>
        <div id="collapseHint_2" class="collapse" aria-labelledby="headingThree" data-parent="#accordion">
            <div class="card-body">
Чтобы создать такие столбцы, лучше всего использовать собственную функцию. Эту функцию можно применить к двум столбцам сразу. 
            </div>
        </div>
    </div>
</div>

In [ ]:
df

In [ ]:
def srez(i):
    
    if isinstance(i,str) and "," in i:
        
        index = i.find(',')
        
        return i[:index]  
    return i
    

In [ ]:
df['director_first_name'] = df['director'].apply(srez)

In [ ]:
df['first_ganre'] = df['genres'].apply(srez)

In [ ]:
df

- Посчитайте, какую долю от общего бюджета фильма составляет государственная поддержка.

In [ ]:
df['share_of_support'] = (df['refundable_support']+df['nonrefundable_support'])/df['budget']

In [ ]:
df.query('budget.isna() != True').head()

### Шаг 3. Проведите исследовательский анализ данных


- Посмотрите, сколько фильмов выходило в прокат каждый год. Обратите внимание, что данные о прокате в кинотеатрах известны не для всех фильмов. Посчитайте, какую долю составляют фильмы с указанной информацией о прокате в кинотеатрах.

In [ ]:
rental_year = df.query('box_office.isna()==False').pivot_table(index='year',values=['box_office'],aggfunc='count')

In [ ]:
rental_year.plot(kind='bar', grid=True,figsize=(20,10),title = 'Количество фильмов, выходивших в прокат по годам, имеющие информацию о прокате')


с каждым годом фильмов выходит в прокат все больше, в 2017 был спад, но до 2019 тенденчия к росту сохраняется

In [ ]:
# Доля фильмов с информацией о прокате 
ratio_films = len(df.query('box_office.isna()==False'))/len(df)
print(f'Доля фильмов с информацией о прокате: {ratio_films:.2f}')

- Изучите, как менялась динамика проката по годам. В каком году сумма сборов была минимальной? А максимальной?

In [ ]:
box_office = df.pivot_table(index='year',values=['box_office'],aggfunc='count')

In [ ]:
 box_office.plot(kind='bar', grid=True,figsize=(20,10),title = 'Количество фильмов, выходивших в прокат по годам')

In [ ]:
sum_boxoffice = df.query('box_office.isna()==False').pivot_table(index = 'year', values= 'box_office',aggfunc='sum')

In [ ]:
sum_boxoffice # Выведем суммарные сборы по годам

In [ ]:
sum_boxoffice.plot(kind='bar',grid=True, figsize=(20,10),title = 'Суммарные сборы фильмов, выходивших в прокат по годам')

In [ ]:
sum_boxoffice.describe()

49,622,695,308.8 - Максимальные сборы 

2,425,499.0 - Минимальные сборы

- С помощью сводной таблицы посчитайте среднюю и медианную сумму сборов для каждого года.

In [ ]:
df.pivot_table(index='year', values='box_office', aggfunc=('mean','median'))

- Определите, влияет ли возрастное ограничение аудитории («6+», «12+», «16+», «18+» и т. д.) на сборы фильма в прокате в период с 2015 по 2019 год? Фильмы с каким возрастным ограничением собрали больше всего денег в прокате? Меняется ли картина в зависимости от года? Если да, предположите, с чем это может быть связано.

In [ ]:
boxoffice_by_category = df.query('(year in [2015,2016,2017,2018,2019]) and (box_office.isna()==False)').pivot_table(index='age_restriction', values='box_office', aggfunc='sum').sort_values(by ='box_office')

In [ ]:
boxoffice_by_category # Суммарные сборы в период с 2015 по 2019 в зависимости от возрастных рейтингов

In [ ]:
boxoffice_by_category.plot(kind='bar',grid=True, figsize=(20,10),title = 'Зависимость кассовых сборов от возрастного рейтинка в период с 2015 по 2019 год')

Самые кассовые фильмы в заданный период имеют возрастное ограничение "16+"

На втором месте "12+"

на третьем с небольшим отставанием "6+"

In [ ]:
boxoffice_by_category_and_year = df.query('(year in [2015,2016,2017,2018,2019]) and (box_office.isna()==False)')\
                                    .pivot_table(index=['year','age_restriction'], values='box_office', aggfunc='sum')

In [ ]:
boxoffice_by_category_and_year

In [ ]:
boxoffice_by_category_and_year.plot(kind='bar',grid=True, figsize=(20,10),title = 'Зависимость кассовых сборов от возрастного рейтинка по годам')

*Для 2015:*
1-е место рейтинг "12+"
1-е место рейтинг "16+"
1-е место рейтинг "6+"

*Для 2016:*
1-е место рейтинг "16+"
2-е место рейтинг "12+" и "6+" с одинаковыми сборами

*Для 2017:*
1-е место рейтинг "16+" (это наибольший показатель за весь период)
2-е место рейтинг "6+"
3-е место рейтинг "18+" (Впервые в тройку вошла эта категория)

*Для 2018:*
1-е место рейтинг "16+"
2-е место рейтинг "12+"
3-е место рейтинг "6+"

*Для 2019:*
Для этого года примерно одинкаовые сборы у категорий: "12+","16+","6+"

Во все года нименьшие сборы с большим отрывом собирает категория "0+"

##### Результаты исследовательского анализа
Для всех годов по сборам самая популярная категория "16+" и аутсайдер "0+"

Самый прибыльный год бы 2017. До 2017 года наблюдался медианных сборов за год, после 2017 прибыль пошла на спад.

Среднее значение сборов по годам сильно завышены из за выбросов

49,622,695,308.8 - Максимальные сборы

2,425,499.0 - Минимальные сборы

Доля фильмов с информацией о прокате: 0.44

### Шаг 4. Исследуйте фильмы, которые получили государственную поддержку

На этом этапе нет конкретных инструкций и заданий — поищите интересные закономерности в данных. Посмотрите, сколько выделяют средств на поддержку кино. Проверьте, хорошо ли окупаются такие фильмы, какой у них рейтинг.

In [ ]:
gos_films = df.query('budget.isna() != True').copy()

In [ ]:
gos_films.head()

In [ ]:
gos_films['support'] = gos_films['refundable_support']+gos_films['nonrefundable_support']

In [ ]:
gos_films['support'].describe()

In [ ]:
gos_films['support'].hist(bins=10, figsize = (20,10), range=(3000000, 500000000), legend=True)

In [ ]:
def category_of_support(i):
    
    if i <= 100000000:
        return 'Категория до 100млн'
    elif 100000000< i <= 200000000:
        return 'Категория до 200млн'
    elif 200000000< i <= 300000000:
        return 'Категория до 300млн'
    elif 300000000< i <= 400000000:
        return 'Категория до 400млн'
    elif 400000000< i <= 500000000:
        return 'Категория до 500млн'
    
 

In [ ]:
gos_films['category_of_support'] = gos_films['support'].apply(category_of_support)

In [ ]:
gos_films['category_of_support'].hist(bins=5, figsize = (20,10), legend=True)

In [ ]:
gos_films.query('category_of_support == "Категория до 500млн"')

Наибольшее количество фильмов с господдержкой до 100 млн рублей.

Минимальная поддержка составляет 3 млн, а максимальная 500млн.

Чаще всего встречается господдержка в размере 35 млн.

75% выборки с господдержкой меньше 60 млн.

In [ ]:
df['ratings'].describe() #  посмотрим как распределяется рейтинг по всем фильмам

Распределим по категиям рейтинг:
25% фильмов с рейтингов ниже 5.8, присвоим таким фильмам категорию "Низкий рейтинг"

50% фильмов с рейтингов ниже 6.6, присвоим  фильмам с рейтингом от 5.8 до 6.6 категорию "Средний рейтинг"    
    
25% фильмов с рейтингов выше 7.2, присвоим таким фильмам категорию "Высокий рейтинг"

In [ ]:
def category_by_rating (i):
    if i <= 5.8:
        return 'Низкий рейтинг'
    elif 5.8 < i <= 6.6:
        return 'Средний рейтинг'
    
    return 'Высокий рейтинг'
    

In [ ]:
gos_films['category_by_rating'] = gos_films['ratings'].apply(category_by_rating)

In [ ]:
gos_films_category = gos_films.pivot_table(index = ['category_by_rating','category_of_support'], values = 'puNumber' , aggfunc = 'count').sort_values(by='puNumber', ascending = False).copy()

In [ ]:
gos_films_category

In [ ]:

gos_films_category.plot(kind='bar',grid=True, figsize=(20,10),title = 'Количество фильмов с господдержкой по категориям рейтинга и размера господдержки')

Катагория до 100млн лидирует по всем трем категориям рейтинга: 

На первом месте "Низкий рейтинг", На втором "Высокий рейтинг" и на третьем - "Средний рейтинг"



Приведем критерии по которым будем оценивать влияние факторов друг на друга:
>Корреляция по критерию Пирсона:
>>слабая -- 0,1-0,3

>>умереная -- 0,3-0,5

>>заметная -- 0,5-0,7

>>высокая -- 0,7-0,9

>>весьма высока -- 0,9-1,0




In [ ]:
gos_films.corr()

In [ ]:
 gos_films.plot(x='budget', y='box_office', kind='scatter')


корреляция бюджета и кссовых сборов умеренная положительная (0,4)

In [ ]:
 gos_films.plot(x='ratings', y='box_office', kind='scatter')

корреляция рейтинга и кссовых сборов слабая положительная (0,2)

In [ ]:
 gos_films.plot(x='support', y='box_office', kind='scatter')

корреляция поддержки и кссовых сборов умеренная положительная (0,5)

In [ ]:
 gos_films.plot(x='budget', y='year', kind='scatter')

корреляция бюджета и года умеренная слабая положительная (0,2)

In [ ]:
# создадим отфильтрованный датасет
gos_films_wonan = gos_films.query('(budget.isna() != True)and(box_office.isna() == False)and(budget != 0)').copy()
# посчитаем окупаемость фильма и добавим ее в новый столбец
gos_films_wonan['payback'] = gos_films_wonan['box_office']/gos_films_wonan['budget']

In [ ]:
len(gos_films_wonan)# посмотрим сколько фильмов попало в выборку после фильтрации

In [ ]:
#
gos_films_wonan.query('payback >= 1').groupby('category_of_support')['payback'].count().plot(kind='bar', figsize = (20,10))

Окупаемость у фильмов с господдержкой не большая (всего 65 фильмов из 300)

Из них 50 это фильмы с господдержкой до 100 млн рублей.

##### Результаты исследования фильмов получивших господдержку

Наибольшее количество фильмов с господдержкой до 100 млн рублей.

Минимальная поддержка составляет 3 млн, а максимальная 500млн.

Чаще всего встречается господдержка в размере 35 млн.

75% выборки с господдержкой меньше 60 млн

Катагория до 100млн лидирует по всем трем категориям рейтинга:

На первом месте "Низкий рейтинг", На втором "Высокий рейтинг" и на третьем - "Средний рейтинг"

Сильных корреляций между данными нет, наивысший показатель зависимости (0,5) был выявлен у пары "support"-"box_office"

### Шаг 5. Напишите общий вывод

Для мирового кино :

49,622,695,308.8 - Максимальные сборы

2,425,499.0 - Минимальные сборы

по сборам самая популярная категория "16+" 

Самый прибыльный год бы 2017

Для российского кино:

Сложно говорить о трендах конкретно для фильмов с господдержкой, так как выборка составляет всего 300 фильмов. 

господдержка:

Наибольшее количество фильмов с господдержкой до 100 млн рублей.

Минимальная поддержка составляет 3 млн, а максимальная 500млн

Чаще всего встречается господдержка в размере 35 млн
